In [ ]:
import tarfile

# Path to your .tgz file
tgz_file = "/content/data-gouv-pdf-txt.tgz"

# Open the tgz file
with tarfile.open(tgz_file, "r:gz") as tar:
    # List all files inside
    print("Files in the archive:")
    for member in tar.getmembers():
        print(member.name)

    # Extract all files
    tar.extractall(path="extracted_data")
    print("Files extracted to 'extracted_data/'")


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
data_gouv_txt/Commune_de_Champagne-sur-Seine/5caef99d634f41038aeaaf53--8701745c-6124-4db5-9e46-ac82e5dd2b53.txt
data_gouv_txt/Commune_de_Champagne-sur-Seine/5bb4c56c634f410ed0fc9482--b31143b0-29f8-4f10-ae42-bc3943e6e985.txt
data_gouv_txt/Commune_de_Champagne-sur-Seine/5bb4c747634f4113f8704649--f7f7c125-10a7-436e-8b4a-5a5f68777851.txt
data_gouv_txt/Commission_Nationale_des_Comptes_de_Campagne_et_des_Financements_Politiques_CNCCFP
data_gouv_txt/Commission_Nationale_des_Comptes_de_Campagne_et_des_Financements_Politiques_CNCCFP/53699158a3a729239d203c2f--90d12938-267a-4bdd-aa20-a8ad5e38850f.txt
data_gouv_txt/Commission_Nationale_des_Comptes_de_Campagne_et_des_Financements_Politiques_CNCCFP/53699158a3a729239d203c2f--d8e2bd8f-3edc-458b-b66e-599444985bb3.txt
data_gouv_txt/Commission_Nationale_des_Comptes_de_Campagne_et_des_Financements_Politiques_CNCCFP/53699158a3a729239d203c2f--c84de5c3-031b-48e1-8e8b-8994209bf801.t

/tmp/ipython-input-2192460939.py:14: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path="extracted_data")


Files extracted to 'extracted_data/'


In [ ]:
!pip install PyPDF2 faiss-cpu sentence-transformers transformers torch



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 48.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "flexudy/t5-small-wav2vec2-grammar-fixer"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.01k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/134 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie shared.weight to encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie shared.weight to decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


In [ ]:
def correct_text(text):
    input_text = "grammar: " + text
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True)

    outputs = model.generate(
        **inputs,
        max_length=256,
        num_beams=4,
        early_stopping=True
    )

    corrected = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected


In [ ]:
!pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.2 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader


In [ ]:
import os
from PyPDF2 import PdfReader

input_folder = "extracted_data"
output_folder = "corrected_data"

os.makedirs(output_folder, exist_ok=True)

def read_file(file_path):
    if file_path.endswith(".txt"):
        with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()

    elif file_path.endswith(".pdf"):
        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() or ""
        return text

    return None


In [ ]:
for root, _, files in os.walk(input_folder):
    for file in files:
        file_path = os.path.join(root, file)
        text = read_file(file_path)

        if text:
            print("Processing:", file)

            corrected = correct_text(text)

            output_path = os.path.join(output_folder, file + ".corrected.txt")
            with open(output_path, "w", encoding="utf-8") as f:
                f.write(corrected)

print("Then correct all files ✔")


Then correct all files ✔


In [ ]:
def split_text(text, max_chars=500):
    chunks = []
    for i in range(0, len(text), max_chars):
        chunks.append(text[i:i+max_chars])
    return chunks


In [ ]:
def correct_long_text(text):
    parts = split_text(text)
    corrected_parts = []

    for part in parts:
        corrected_parts.append(correct_text(part))

    return " ".join(corrected_parts)


In [ ]:
processed_files = []
failed_files = []
total_files = 0

for root, _, files in os.walk(input_folder):
    for file in files:
        total_files += 1

print("Total files:", total_files)
print("-" * 40)

counter = 0

for root, _, files in os.walk(input_folder):
    for file in files:
        counter += 1
        file_path = os.path.join(root, file)

        try:
            text = read_file(file_path)

            if text and len(text.strip()) > 0:
                print(f"[{counter}/{total_files}] Processing:", file)

                corrected = correct_long_text(text)

                output_path = os.path.join(output_folder, file + ".corrected.txt")
                with open(output_path, "w", encoding="utf-8") as f:
                    f.write(corrected)

                processed_files.append(file)
            else:
                print("Skipped (empty):", file)

        except Exception as e:
            print("Error processing:", file)
            failed_files.append(file)

print("\n✔ Correction completed")


Total files: 0
----------------------------------------

✔ Correction completed


In [ ]:
report_path = os.path.join(output_folder, "report.txt")

with open(report_path, "w", encoding="utf-8") as f:
    f.write("FILES PROCESSED:\n")
    for name in processed_files:
        f.write(name + "\n")

    f.write("\nFAILED FILES:\n")
    for name in failed_files:
        f.write(name + "\n")

print("Then save the report ✔")


Then save the report ✔


In [ ]:
!pip install gradio transformers torch sentencepiece PyPDF2


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "vennify/t5-base-grammar-correction"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/260 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie shared.weight to encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie shared.weight to decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


In [ ]:
def split_text(text, max_chars=500):
    return [text[i:i+max_chars] for i in range(0, len(text), max_chars)]

def correct_text(text):
    parts = split_text(text)
    corrected_parts = []

    for part in parts:
        input_text = "grammar: " + part
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True)

        outputs = model.generate(
            **inputs,
            max_length=512,
            num_beams=4,
            early_stopping=True
        )

        corrected_parts.append(
            tokenizer.decode(outputs[0], skip_special_tokens=True)
        )

    return " ".join(corrected_parts)


In [ ]:
import gradio as gr

interface = gr.Interface(
    fn=correct_text,
    inputs=gr.Textbox(lines=6, placeholder="Enter text here..."),
    outputs=gr.Textbox(label="Corrected text"),
    title="Spelling and Grammar Corrector ",
    description="Enter a text and it will be automatically corrected "
)

interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cdc72640071ac15661.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
